In [1]:
from datetime import datetime
from ripe.atlas.cousteau import AtlasRequest, AtlasResultsRequest
from ripe.atlas.sagan import Result, DnsResult

from ipwhois import IPWhois
STOPPED = 4


def get_measurements_collection_by_date_and_type(date, measurement_type):
    timestamp = datetime.strptime(date, "%Y-%m-%d").timestamp()
    filters = {"type": measurement_type,
               "status": STOPPED,
               "start_time__gte": timestamp,
               "start_time__lt": timestamp+24*60*60,
               "fields": "id,start_time,stop_time,af,query_argument,query_type,result"
               }
    url_path = "/api/v2/measurements/"
    is_success, response = AtlasRequest(
        **{"url_path": url_path}).get(**filters)
    if is_success:
        return response["results"]
    else:
        return None


def get_measurement_results(measurement_id):
    url_path = f"/api/v2/measurements/{measurement_id}/results/"
    is_success, response_results = AtlasRequest(
        **{"url_path": url_path}).get()
    if is_success:
        return response_results
    else:
        return None


def get_probes_geolocation_by_id_list(probe_id_list):
    url_path = "/api/v2/probes/"
    result = []
    for sub_list in [probe_id_list[i:i+500] for i in range(0, len(probe_id_list), 500)]:
        is_success, response_results = AtlasRequest(
            **{"url_path": url_path}).get(**{"id__in": ",".join(sub_list)})
        if is_success:
            result.extend(response_results["results"])
        else:
            continue
    return result


def map_measurement_result_to_probe_info(parsed_result):
    return {
        "probe_id": parsed_result.probe_id,
        "rtt": parsed_result.responses[0].response_time,
    }


def create_measurement_hash(measurement_id):
    
    def find_object_by_id(collection, id):
        for obj in collection:
            if obj["id"] == id:
                return obj  # Return the object if found
            
    meausrement_results = get_measurement_results(measurement_id)
    probes_information = [
        map_measurement_result_to_probe_info(DnsResult(result))
        for result in meausrement_results if not DnsResult(result).is_error]
    probes_geolocation = get_probes_geolocation_by_id_list(
        [str(probe_obj["probe_id"]) for probe_obj in probes_information])
    for probe_info in probes_information:
        according_probe_geolocation = find_object_by_id(
            probes_geolocation, probe_info["probe_id"])
        probe_info


def check_dns_measurements(date):
    measurements = get_measurements_collection_by_date_and_type(date, "dns")
    for measurement in measurements:
        print('Measurement ID:', measurement.get("id"))
        print('Start Time:', datetime.fromtimestamp(
            measurement.get("start_time")))
        print('Stop Time:', datetime.fromtimestamp(
            measurement.get("stop_time")))
        print(
            f"Query: {measurement.get('af')} {measurement.get('query_argument')} {measurement.get('query_type')}")
        print('Results Link:', measurement.get("result"))
        print('----')
        measurement_hash = create_measurement_hash(measurement.get("id"))
        print()
        # meausrement_results = get_measurement_results(measurement.get('id'))
        # probes_information = [
        #     map_measurement_result_to_probe_info(DnsResult(result))
        #     for result in meausrement_results if not DnsResult(result).is_error]

        # probes_geolocation=get_probes_geolocation_by_id_list([str(probe_obj["probe_id"]) for probe_obj in probes_information])
        # for res in meausrement_results:
        #     parsed_result = DnsResult(res)
        #     if not parsed_result.is_error:
        #         is_success, results = AtlasRequest(
        #             **{"url_path": f"/api/v2/probes/{parsed_result.probe_id}/"}).get()
        #         print('DNS Response Time:',
        #               parsed_result.responses[0].response_time)
        #         print('Origin IP:', parsed_result.origin)
        #         # print('DNS Response Time:', parsed_result.response_time)
        #         print('DNS Result:', parsed_result.result)
        #         print('----')
        # else:
        #     print('Failed to get the results for the measurement with ID:', result.get("id"))


check_dns_measurements("2022-05-01")

Measurement ID: 40658914
Start Time: 2022-05-01 01:03:37
Stop Time: 2022-05-01 01:10:04
Query: 4 loc.united.test. TXT
Results Link: https://atlas.ripe.net/api/v2/measurements/40658914/results/
----



Measurement ID: 40658955
Start Time: 2022-05-01 01:05:11
Stop Time: 2022-05-01 01:15:03
Query: 4 redirector.googlevideo.com. A
Results Link: https://atlas.ripe.net/api/v2/measurements/40658955/results/
----



Measurement ID: 40659351
Start Time: 2022-05-01 01:25:57
Stop Time: 2022-05-01 01:35:03
Query: 4 loc.united.test. TXT
Results Link: https://atlas.ripe.net/api/v2/measurements/40659351/results/
----



Measurement ID: 40659832
Start Time: 2022-05-01 01:46:39
Stop Time: 2022-05-01 01:55:02
Query: 6 loc.united.test. TXT
Results Link: https://atlas.ripe.net/api/v2/measurements/40659832/results/
----


In [10]:

from pprint import pprint
from ipwhois.experimental import bulk_lookup_rdap,get_bulk_asn_whois
ip_list = ['74.125.225.229', '2001:4860:4860::8888', '62.239.237.1', '2a00:2381:ffff::1', '210.107.73.73',
        #    '2001:240:10c:1::ca20:9d1d', '200.57.141.161', '2801:10:c000::', '196.11.240.215', '2001:43f8:7b0::', '133.1.2.5', '115.1.2.3'
           ]
# obj = IPWhois('["8.8.4.4", "1.1.1.1", "2c0f:fb50:4003::"]')
results= get_bulk_asn_whois(addresses=ip_list)
# results, stats = bulk_lookup_rdap(addresses=ip_list)
pprint(results)

('Bulk mode; whois.cymru.com [2023-06-05 05:36:02 +0000]\n'
 '15169   | 74.125.225.229   | 74.125.225.0/24     | US | arin     | '
 '2007-03-13 | GOOGLE, US\n'
 '15169   | 2001:4860:4860::8888                     | 2001:4860::/32      | '
 'US | arin     | 2005-03-14 | GOOGLE, US\n'
 '2856    | 62.239.237.1     | 62.239.0.0/16       | GB | ripencc  | '
 '2001-01-02 | BT-UK-AS BTnet UK Regional network, GB\n'
 '2856    | 2a00:2381:ffff::1                        | 2a00:2380::/25      | '
 'GB | ripencc  | 2007-08-29 | BT-UK-AS BTnet UK Regional network, GB\n'
 '3786    | 210.107.73.73    | 210.107.0.0/17      | KR | apnic    | '
 '1997-08-29 | LGDACOM LG DACOM Corporation, KR\n')
